In [ ]:
# Utility imports
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import pykep as pk
import MSMatch as mm
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
cfg=mm.load_cfg(None)
cfg.t0 = pk.epoch_from_string(cfg.start_time)  # starting date of our simulation
cfg.nodes = 8
# get constellations params
# PASEOS setup
altitude = 786 * 1000  # altitude above the Earth's ground [m]
inclination = 98.62  # inclination of the orbit
nPlanes = 1 # the number of orbital planes (see linked wiki article)
nSats = cfg.nodes # the number of satellites per orbital plane

# Compute orbits of LEO satellites
planet_list,sats_pos_and_v = mm.get_constellation(altitude,inclination,nSats,nPlanes,cfg.t0)

In [ ]:
logger_level = "INFO"
logger = mm.get_logger(cfg.save_dir, logger_level)

In [ ]:
from mpi4py import MPI
nodes,actors = [],[]
for rank in range(cfg.nodes):
    comm = MPI.COMM_WORLD
    node = mm.SpaceCraftNode(sats_pos_and_v[rank], cfg, None, rank, comm)
    nodes.append(node)
    actors.append(node.paseos.local_actor)

In [ ]:
positions = []
for actor in actors:
    positions.append(actor.get_position(cfg.t0))

In [ ]:
# Ground stations
from paseos import ActorBuilder, GroundstationActor,SpacecraftActor
stations = [
    ["Maspalomas", 27.7629, -15.6338, 205.1],
    ["Matera", 40.6486, 16.7046, 536.9],
    ["Svalbard", 78.9067, 11.8883, 474.0],
]
groundstation_actors = []
for station in stations:
    gs_actor = ActorBuilder.get_actor_scaffold(
        name=station[0], actor_type=GroundstationActor, epoch=cfg.t0
    )
    ActorBuilder.set_ground_station_location(
        gs_actor,
        latitude=station[1],
        longitude=station[2],
        elevation=station[3],
        minimum_altitude_angle=5,
    )
    # paseos_instance.add_known_actor(gs_actor)
    groundstation_actors.append(gs_actor)

In [ ]:
geosat = ActorBuilder.get_actor_scaffold("EDRS-C", SpacecraftActor, epoch=cfg.t0)

altitude =  35786 * 1000  # altitude above the Earth's ground [m]
inclination = 0.  # inclination of the orbit
nPlanes = 1 # the number of orbital planes (see linked wiki article)
nSats = 1 # the number of satellites per orbital plane
t0 = cfg.t0

# Compute orbits of LEO satellites
planet_list,sats_pos_and_v = mm.get_constellation(altitude,inclination,nSats,nPlanes,t0,startingW=31)
earth = pk.planet.jpl_lp("earth")
ActorBuilder.set_orbit(geosat,sats_pos_and_v[0][0],sats_pos_and_v[0][1],t0,earth)

In [ ]:
tX,tY,tZ = [],[],[]
N = 600
for n in range(N):
    pos = actors[0].get_position(pk.epoch(t0.mjd2000 + pk.SEC2DAY * n * 10))
    tX.append(pos[0])
    tY.append(pos[1])
    tZ.append(pos[2])

In [ ]:
with plt.style.context("dark_background"):
    ax = plt.figure(figsize=(4, 4),dpi=300).add_subplot(111,projection='3d',facecolor="black",computed_zorder=False)
    ax.view_init(elev=0, azim=300)
    
    # Get rid of the panes
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

    # Get rid of the spines
    ax.xaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    ax.get_zaxis().set_ticks([])

    # load bluemarble with PIL
    import PIL
    R = 6371000
    stride = 4
    offset = 3e5
    offset = 1e6

    if offset == 1e6:
        fontsize = 10
    else:
        fontsize = 12

    bm = PIL.Image.open('bluemarble_med.png')
    # it's big, so I'll rescale it, convert to array, and divide by 256 to get RGB values that matplotlib accept 
    # bm = np.array(bm.resize([int(d) for d in bm.size]))/256.
    bm = np.array(bm) / 256

    # coordinates of the image - don't know if this is entirely accurate, but probably close
    lons = np.linspace(-180, 180, bm.shape[1]) * np.pi/180 
    lats = np.linspace(-90, 90, bm.shape[0])[::-1] * np.pi/180 

    x = np.outer(np.cos(lons+30.45), np.cos(lats)).T
    y = np.outer(np.sin(lons+30.45), np.cos(lats)).T
    z = np.outer(np.ones(np.size(lons)), np.sin(lats)).T
    ax.plot_surface(R*x, R*y, R*z, rstride=stride, cstride=stride, 
                    facecolors = bm,linewidth=0,alpha=1.0,zorder=0)

    pos = geosat.get_position(t0)
    ax.scatter(pos[0],pos[1],pos[2],color="cyan",marker="d",zorder=20)
    # Write text labels
    ax.text(pos[0]+offset,pos[1],pos[2],geosat.name,fontsize=fontsize,ha="left",va="center",zorder=30)

    for pos,actor in zip(positions,actors):
        ax.scatter(pos[0],pos[1],pos[2],color="white",marker="d", s=2).set_zorder(10)
        ax.text(pos[0]+offset,pos[1],pos[2],actor.name,fontsize=fontsize,ha="left",va="center")

    # for gs in groundstation_actors:
    #     pos = gs.get_position(t0)
    #     ax.scatter(pos[0],pos[1],pos[2],color="red",zorder=20)
    #     # Write text labels
    #     ax.text(pos[0]+offset,pos[1],pos[2],gs.name,fontsize=fontsize,ha="left",va="center",zorder=30)


    if offset == 1e6:
        ax.set_xlim(1.5e7,1.55e7)
        ax.set_ylim(1.5e7,1.55e7)
        ax.set_zlim(-1.0e7,1.55e7)
    else:
        ax.set_xlim(-4.5e6,4.5e6)
        ax.set_ylim(-4.5e6,4.5e6)
        ax.set_zlim(-4.5e6,4.5e6)

    ax.plot(tX,tY,tZ,zorder=10,linestyle="--",linewidth=1,alpha=0.5)
    ax.set_aspect("equal")

    

     # Write text labels
    plt.annotate(t0,xy=(0.01, 0.01),xycoords="figure fraction",)
    plt.annotate(f"t={nodes[0].paseos._state.time:<10.2e}",xy=(0.99, 0.01),xycoords="figure fraction",horizontalalignment="right",zorder=30)

    plt.show()